# Customizing Colorbars

Plot legends identify discrete labels of discrete points. For continuous labels based on the color points, lines, or regions, a labeled colorbar can be a great tool. In MPL, a colorbar is a separate axis that cna provide a key for them meaning of colors in a plot. 

In [2]:
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np